# QiCore CLI - Framework and State Management

**Interactive tutorial for building sophisticated CLI applications with state machines**

## What You'll Learn

- ✅ **State machine architecture** - XState-powered hierarchical states
- ✅ **Command system design** - CLI commands vs Agent commands
- ✅ **Message-driven integration** - CLI ↔ AMSG ↔ Agent workflows
- ✅ **UI framework adapters** - Readline, Ink, and Hybrid approaches
- ✅ **Production patterns** - Hotkeys, progress display, error handling

---

## 📚 Prerequisites

This tutorial builds on all previous modules:
- **[01-qi-base.ipynb](./01-qi-base.ipynb)** - Result<T> patterns
- **[02-qi-core.ipynb](./02-qi-core.ipynb)** - Infrastructure services
- **[03-qi-amsg.ipynb](./03-qi-amsg.ipynb)** - Message-driven architecture

Let's start by setting up our imports and understanding the CLI architecture:

In [ ]:
// Import Result<T> fundamentals from @qi/base
import { 
  success, failure, match, map, flatMap,
  type Result, type QiError,
  validationError, systemError, networkError
} from '@qi/base'

// Import message system from @qi/amsg
import { 
  QiAsyncMessageQueue,
  QiMessageFactory,
  MessageType, MessagePriority
} from '@qi/amsg'

import type { 
  QiMessage,
  UserInputMessage,
  CommandMessage,
  SystemControlMessage
} from '@qi/amsg'

// CLI framework types and interfaces (conceptual - @qi/cli is in development)
// In the actual implementation, these would be imported from '@qi/cli'

// State management types (conceptual XState 5 integration)
type AppState = 'busy' | 'ready'
type AppSubState = 'planning' | 'editing' | 'generic'
type StateEvent = 
  | { type: 'START_TASK'; taskName: string }
  | { type: 'TASK_COMPLETE' }
  | { type: 'TASK_ERROR'; error?: string }
  | { type: 'CYCLE_STATES' }
  | { type: 'SET_STATE'; subState: AppSubState }
  | { type: 'RESET' }

// CLI command types
interface CLICommand {
  readonly name: string
  readonly description: string
  readonly usage: string
  readonly aliases?: readonly string[]
}

interface CLICommandResult {
  readonly success: boolean
  readonly output: string
  readonly shouldExit?: boolean
  readonly shouldClear?: boolean
}

// Display welcome message
Deno.jupyter.html`
<div style="background: linear-gradient(135deg, #6c5ce7 0%, #a29bfe 100%); color: white; padding: 20px; border-radius: 10px; margin: 15px 0;">
  <h3 style="margin: 0 0 15px 0;">🖥️ QiCore CLI Tutorial Started!</h3>
  <p style="margin: 0;">Ready to explore sophisticated CLI architecture with state management and message integration!</p>
</div>
`

console.log('✅ CLI framework imports and types ready!')
console.log('✅ Ready to explore CLI architecture patterns!')

## 1. State Machine Architecture

The CLI uses XState 5 for hierarchical state management. Let's understand the state structure:

In [ ]:
console.log('=== CLI State Machine Architecture ===\n')

// Mock state manager implementation (conceptual)
class CLIStateManager {
  private currentState: AppState = 'ready'
  private currentSubState: AppSubState = 'generic'
  private taskName: string | null = null
  private sessionId: string
  private subscribers = new Set<(state: AppState, subState: AppSubState) => void>()
  
  constructor() {
    this.sessionId = `session_${Date.now()}_${Math.random().toString(36).substr(2, 8)}`
    console.log(`🔄 State manager initialized (Session: ${this.sessionId})`)
  }
  
  // State access methods
  getCurrentState(): AppState {
    return this.currentState
  }
  
  getCurrentSubState(): AppSubState {
    return this.currentSubState
  }
  
  getCurrentTaskName(): string | null {
    return this.taskName
  }
  
  getSessionId(): string {
    return this.sessionId
  }
  
  // State transitions
  transition(event: StateEvent): void {
    const oldState = this.currentState
    const oldSubState = this.currentSubState
    
    console.log(`\n🔄 State transition: ${event.type}`)
    console.log(`   From: ${oldState}${oldState === 'ready' ? `.${oldSubState}` : ''}`)
    
    switch (event.type) {
      case 'START_TASK':
        if (this.currentState === 'ready') {
          this.currentState = 'busy'
          this.taskName = event.taskName
          console.log(`   Task: ${event.taskName}`)
        } else {
          console.log(`   ⚠️ Cannot start task - already busy with: ${this.taskName}`)
          return
        }
        break
        
      case 'TASK_COMPLETE':
      case 'TASK_ERROR':
        if (this.currentState === 'busy') {
          this.currentState = 'ready'
          console.log(`   Completed task: ${this.taskName}`)
          this.taskName = null
          if (event.type === 'TASK_ERROR' && event.error) {
            console.log(`   ❌ Task error: ${event.error}`)
          }
        } else {
          console.log(`   ⚠️ No active task to complete`)
          return
        }
        break
        
      case 'CYCLE_STATES':
        if (this.currentState === 'ready') {
          this.currentSubState = this.getNextSubState(this.currentSubState)
          console.log(`   🔄 Cycled to: ${this.currentSubState}`)
        } else {
          console.log(`   ⚠️ Cannot cycle states - currently busy`)
          return
        }
        break
        
      case 'SET_STATE':
        if (this.currentState === 'ready') {
          this.currentSubState = event.subState
          console.log(`   ➡️ Set to: ${event.subState}`)
        } else {
          console.log(`   ⚠️ Cannot set state - currently busy`)
          return
        }
        break
        
      case 'RESET':
        this.currentState = 'ready'
        this.currentSubState = 'generic'
        this.taskName = null
        console.log(`   🔄 Reset to: ready.generic`)
        break
    }
    
    console.log(`   To: ${this.currentState}${this.currentState === 'ready' ? `.${this.currentSubState}` : ''}`)
    
    // Notify subscribers
    this.subscribers.forEach(callback => {
      try {
        callback(this.currentState, this.currentSubState)
      } catch (error) {
        console.error('State subscription error:', error)
      }
    })
  }
  
  private getNextSubState(current: AppSubState): AppSubState {
    // Cycle: generic → planning → editing → generic
    switch (current) {
      case 'generic': return 'planning'
      case 'planning': return 'editing'
      case 'editing': return 'generic'
      default: return 'generic'
    }
  }
  
  // Subscription management
  subscribe(callback: (state: AppState, subState: AppSubState) => void): () => void {
    this.subscribers.add(callback)
    return () => this.subscribers.delete(callback)
  }
  
  // Convenience methods
  setBusy(taskName: string): void {
    this.transition({ type: 'START_TASK', taskName })
  }
  
  setReady(): void {
    this.transition({ type: 'TASK_COMPLETE' })
  }
  
  cycleReadyStates(): void {
    this.transition({ type: 'CYCLE_STATES' })
  }
  
  // Display current state info
  displayState(): void {
    const stateEmoji = this.currentState === 'busy' ? '⚙️' : this.getSubStateEmoji()
    const stateLabel = this.currentState === 'busy' 
      ? `Processing: ${this.taskName}` 
      : this.getSubStateLabel()
      
    console.log(`\n📊 Current State: ${stateEmoji} ${stateLabel}`)
    console.log(`   Session: ${this.sessionId}`)
  }
  
  private getSubStateEmoji(): string {
    switch (this.currentSubState) {
      case 'planning': return '🧠'
      case 'editing': return '✏️'
      case 'generic': return '💬'
      default: return '❓'
    }
  }
  
  private getSubStateLabel(): string {
    switch (this.currentSubState) {
      case 'planning': return 'Planning Mode - Strategic thinking and architecture'
      case 'editing': return 'Editing Mode - Code writing and implementation'
      case 'generic': return 'Generic Mode - General conversation and help'
      default: return 'Unknown Mode'
    }
  }
}

// Demonstrate the state machine
console.log('🎬 State Machine Demo\n')

const stateManager = new CLIStateManager()

// Set up state monitoring
const unsubscribe = stateManager.subscribe((state, subState) => {
  console.log(`   📢 State change notification: ${state}${state === 'ready' ? `.${subState}` : ''}`)
})

// Initial state
stateManager.displayState()

// Demonstrate state cycling (Shift+Tab functionality)
console.log('\n--- Demonstrating State Cycling (Shift+Tab) ---')
stateManager.cycleReadyStates()
stateManager.displayState()

stateManager.cycleReadyStates()
stateManager.displayState()

stateManager.cycleReadyStates() // Should cycle back to generic
stateManager.displayState()

// Demonstrate task execution
console.log('\n--- Demonstrating Task Execution ---')
stateManager.setBusy('Building React component')
stateManager.displayState()

// Try to cycle while busy (should be prevented)
console.log('\n--- Trying to Cycle While Busy (Should Fail) ---')
stateManager.cycleReadyStates()

// Complete the task
console.log('\n--- Task Completion ---')
stateManager.setReady()
stateManager.displayState()

// Clean up
unsubscribe()

Deno.jupyter.html`
<div style="background: #e8f5e8; padding: 15px; border-radius: 8px; margin: 15px 0;">
  <h4 style="color: #2d5a2d; margin: 0 0 10px 0;">🔄 State Machine Benefits</h4>
  <ul style="margin: 10px 0; padding-left: 20px; color: #2d5a2d;">
    <li><strong>Predictable behavior</strong> - Well-defined state transitions</li>
    <li><strong>Mode switching</strong> - Different interaction modes for different contexts</li>
    <li><strong>Task management</strong> - Clear busy/ready lifecycle</li>
    <li><strong>User feedback</strong> - Visual indicators for current state</li>
    <li><strong>Keyboard shortcuts</strong> - Shift+Tab cycling between modes</li>
  </ul>
</div>
`

## 2. Command System Design

The CLI framework separates CLI commands (handled locally) from Agent commands (sent via message queue):

In [ ]:
console.log('\n=== Command System Design ===\n')

// CLI Command Handler - handles commands that start with '/'
class SimpleCLICommandHandler {
  private commands = new Map<string, CLICommand>()
  private aliases = new Map<string, string>()
  
  constructor() {
    this.registerBuiltInCommands()
  }
  
  private registerBuiltInCommands(): void {
    const commands: CLICommand[] = [
      {
        name: 'help',
        description: 'Show available CLI commands',
        usage: '/help [command]',
        aliases: ['h', '?']
      },
      {
        name: 'exit',
        description: 'Exit the CLI application',
        usage: '/exit',
        aliases: ['quit', 'q']
      },
      {
        name: 'clear',
        description: 'Clear the terminal screen',
        usage: '/clear',
        aliases: ['cls']
      },
      {
        name: 'version',
        description: 'Show CLI version information',
        usage: '/version',
        aliases: ['v']
      },
      {
        name: 'status',
        description: 'Show current CLI status and state',
        usage: '/status',
        aliases: ['st']
      }
    ]
    
    for (const command of commands) {
      this.commands.set(command.name, command)
      
      if (command.aliases) {
        for (const alias of command.aliases) {
          this.aliases.set(alias, command.name)
        }
      }
    }
    
    console.log(`✅ Registered ${this.commands.size} CLI commands with ${this.aliases.size} aliases`)
  }
  
  isCommand(input: string): boolean {
    return input.trim().startsWith('/')
  }
  
  parseCommand(input: string): { command: string; args: string[]; rawInput: string } | null {
    if (!this.isCommand(input)) return null
    
    const trimmed = input.trim().substring(1) // Remove '/'
    const parts = trimmed.split(/\s+/)
    const command = parts[0] || ''
    const args = parts.slice(1)
    
    return { command, args, rawInput: input }
  }
  
  executeCommand(
    command: string, 
    args: string[], 
    context: { stateManager?: CLIStateManager } = {}
  ): CLICommandResult {
    // Resolve alias to actual command name
    const actualCommand = this.aliases.get(command) || command
    const commandDef = this.commands.get(actualCommand)
    
    if (!commandDef) {
      return {
        success: false,
        output: `❌ Unknown command: /${command}\nType /help for available commands.`
      }
    }
    
    console.log(`\n⚙️ Executing CLI command: /${actualCommand} ${args.join(' ')}`)
    
    // Execute the specific command
    switch (actualCommand) {
      case 'help':
        return this.executeHelp(args)
        
      case 'exit':
        return {
          success: true,
          output: '👋 Goodbye! Thanks for using QiCore CLI.',
          shouldExit: true
        }
        
      case 'clear':
        return {
          success: true,
          output: '\n'.repeat(50), // Simulate clearing screen
          shouldClear: true
        }
        
      case 'version':
        return {
          success: true,
          output: `🚀 QiCore CLI v1.0.0\n   Framework: TypeScript\n   State: XState 5\n   Messages: AMSG`
        }
        
      case 'status':
        return this.executeStatus(context.stateManager)
        
      default:
        return {
          success: false,
          output: `❌ Command '${actualCommand}' not implemented yet.`
        }
    }
  }
  
  private executeHelp(args: string[]): CLICommandResult {
    if (args.length > 0) {
      // Help for specific command
      const commandName = this.aliases.get(args[0]) || args[0]
      const command = this.commands.get(commandName)
      
      if (!command) {
        return {
          success: false,
          output: `❌ Unknown command: ${args[0]}`
        }
      }
      
      let output = `\n📖 Command: /${command.name}\n`
      output += `   Description: ${command.description}\n`
      output += `   Usage: ${command.usage}\n`
      if (command.aliases && command.aliases.length > 0) {
        output += `   Aliases: ${command.aliases.map(a => `/${a}`).join(', ')}\n`
      }
      
      return { success: true, output }
    }
    
    // General help
    let output = '\n📚 Available CLI Commands:\n\n'
    
    for (const [name, command] of this.commands) {
      output += `   /${name.padEnd(8)} - ${command.description}\n`
      if (command.aliases && command.aliases.length > 0) {
        output += `   ${' '.repeat(10)} (aliases: ${command.aliases.map(a => `/${a}`).join(', ')})\n`
      }
    }
    
    output += '\n💡 Tips:\n'
    output += '   - Use /help <command> for detailed help\n'
    output += '   - Commands start with / (slash)\n'
    output += '   - Everything else goes to the AI agent\n'
    output += '   - Press Shift+Tab to cycle between modes\n'
    
    return { success: true, output }
  }
  
  private executeStatus(stateManager?: CLIStateManager): CLICommandResult {
    let output = '\n📊 CLI Status:\n\n'
    
    if (stateManager) {
      const state = stateManager.getCurrentState()
      const subState = stateManager.getCurrentSubState()
      const taskName = stateManager.getCurrentTaskName()
      const sessionId = stateManager.getSessionId()
      
      output += `   Current State: ${state}`
      if (state === 'ready') {
        output += `.${subState}`
      }
      output += '\n'
      
      if (taskName) {
        output += `   Active Task: ${taskName}\n`
      }
      
      output += `   Session ID: ${sessionId}\n`
      
      output += '\n   Mode Descriptions:\n'
      output += '   🧠 Planning  - Strategic thinking and architecture\n'
      output += '   ✏️ Editing   - Code writing and implementation\n'
      output += '   💬 Generic   - General conversation and help\n'
      
    } else {
      output += '   State Manager: Not available\n'
    }
    
    output += '\n   Framework: QiCore CLI v1.0.0\n'
    output += `   Timestamp: ${new Date().toISOString()}\n`
    
    return { success: true, output }
  }
  
  getCommands(): CLICommand[] {
    return Array.from(this.commands.values())
  }
}

// Demonstrate the command system
console.log('🎬 Command System Demo\n')

const commandHandler = new SimpleCLICommandHandler()
const testStateManager = new CLIStateManager()

// Test various command inputs
const testInputs = [
  '/help',
  '/version',
  '/status',
  '/h exit',     // Help for specific command
  '/q',          // Alias test
  '/unknown',    // Unknown command
  'hello world', // Not a command (should go to agent)
]

console.log('--- Testing Command Processing ---\n')

for (const input of testInputs) {
  console.log(`Input: "${input}"`)
  
  if (commandHandler.isCommand(input)) {
    const parsed = commandHandler.parseCommand(input)
    if (parsed) {
      const result = commandHandler.executeCommand(
        parsed.command, 
        parsed.args, 
        { stateManager: testStateManager }
      )
      
      console.log(`Result: ${result.success ? '✅' : '❌'}`)
      console.log(`Output: ${result.output.split('\n')[0]}...`) // First line only for demo
      
      if (result.shouldExit) {
        console.log('   🚪 Would exit CLI')
      }
    }
  } else {
    console.log('   → Would send to Agent via message queue')
  }
  
  console.log() // Blank line
}

Deno.jupyter.html`
<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 15px 0;">
  <h4 style="color: #856404; margin: 0 0 10px 0;">🔀 Command Separation</h4>
  <p style="margin: 0; color: #856404;">
    <strong>CLI Commands</strong> (start with /) are handled locally for immediate response.<br>
    <strong>Agent Commands</strong> (everything else) are sent through the message queue for AI processing.<br>
    This separation keeps the CLI responsive while enabling complex AI interactions.
  </p>
</div>
`

## 3. Message-Driven CLI Integration

Let's see how the CLI integrates with the AMSG system for agent communication:

In [ ]:
console.log('\n=== Message-Driven CLI Integration ===\n')

// Message-driven CLI implementation
class MessageDrivenCLI {
  private messageQueue: QiAsyncMessageQueue<QiMessage>
  private messageFactory: QiMessageFactory
  private stateManager: CLIStateManager
  private commandHandler: SimpleCLICommandHandler
  private isRunning = false
  private sessionId: string
  
  constructor() {
    this.messageQueue = new QiAsyncMessageQueue<QiMessage>({
      maxSize: 100,
      enableStats: true,
      priorityQueuing: true
    })
    this.messageFactory = new QiMessageFactory()
    this.stateManager = new CLIStateManager()
    this.commandHandler = new SimpleCLICommandHandler()
    this.sessionId = `cli_${Date.now()}_${Math.random().toString(36).substr(2, 8)}`
    
    console.log(`🖥️ Message-driven CLI initialized (Session: ${this.sessionId})`)
  }
  
  async start(): Promise<Result<void, QiError>> {
    if (this.isRunning) {
      return failure(systemError('CLI already running'))
    }
    
    this.isRunning = true
    console.log('\n🚀 CLI started and ready for input\n')
    
    return success(undefined)
  }
  
  async processInput(input: string): Promise<Result<void, QiError>> {
    if (!this.isRunning) {
      return failure(systemError('CLI not running'))
    }
    
    console.log(`\n📝 User input: "${input}"`)
    
    // Check if it's a CLI command
    if (this.commandHandler.isCommand(input)) {
      return this.handleCLICommand(input)
    } else {
      return await this.handleAgentInput(input)
    }
  }
  
  private handleCLICommand(input: string): Result<void, QiError> {
    const parsed = this.commandHandler.parseCommand(input)
    if (!parsed) {
      return failure(validationError('Invalid command format'))
    }
    
    console.log(`⚙️ Processing CLI command locally: /${parsed.command}`)
    
    const result = this.commandHandler.executeCommand(
      parsed.command,
      parsed.args,
      { stateManager: this.stateManager }
    )
    
    console.log(`📤 CLI Response: ${result.success ? '✅' : '❌'}`)
    console.log(`   Output: ${result.output.substring(0, 100)}${result.output.length > 100 ? '...' : ''}`)
    
    if (result.shouldExit) {
      this.isRunning = false
      console.log('🚪 CLI exit requested')
    }
    
    if (result.shouldClear) {
      console.log('🧹 Screen clear requested')
    }
    
    return success(undefined)
  }
  
  private async handleAgentInput(input: string): Promise<Result<void, QiError>> {
    console.log('📨 Sending input to agent via message queue')
    
    // Create user input message
    const messageResult = this.messageFactory.createUserInputMessage(
      input,
      'cli',
      false
    )
    
    return match(
      async (message) => {
        // Add session context
        (message as any).sessionId = this.sessionId
        (message as any).cliState = {
          state: this.stateManager.getCurrentState(),
          subState: this.stateManager.getCurrentSubState()
        }
        
        const enqueueResult = this.messageQueue.enqueue(message, MessagePriority.NORMAL)
        
        return match(
          async () => {
            console.log('✅ Message queued for agent processing')
            
            // Update CLI state to show we're waiting for agent
            this.stateManager.setBusy(`Processing: ${input.substring(0, 30)}...`)
            
            return success(undefined)
          },
          async (error) => failure(error),
          enqueueResult
        )
      },
      async (error) => failure(error),
      messageResult
    )
  }
  
  // Simulate agent response processing
  async receiveAgentResponse(response: string): Promise<Result<void, QiError>> {
    console.log('\n🤖 Agent response received:')
    console.log(`   ${response.substring(0, 80)}${response.length > 80 ? '...' : ''}`)
    
    // Update CLI state back to ready
    this.stateManager.setReady()
    
    return success(undefined)
  }
  
  // Handle hotkey inputs
  handleHotkey(key: string): Result<void, QiError> {
    console.log(`\n⌨️ Hotkey pressed: ${key}`)
    
    switch (key) {
      case 'Shift+Tab':
        if (this.stateManager.getCurrentState() === 'ready') {
          this.stateManager.cycleReadyStates()
          console.log(`🔄 Mode switched: ${this.stateManager.getCurrentSubState()}`)
        } else {
          console.log('⚠️ Cannot switch modes while busy')
        }
        break
        
      case 'Escape':
        console.log('🛑 Cancel requested')
        if (this.stateManager.getCurrentState() === 'busy') {
          this.stateManager.setReady()
          console.log('✅ Task cancelled, returned to ready state')
        }
        break
        
      case 'Ctrl+C':
        console.log('🚨 Interrupt signal - sending high priority message')
        const interruptResult = this.messageFactory.createSystemControlMessage(
          'pause',
          true,
          'User interrupt (Ctrl+C)'
        )
        
        return match(
          (message) => {
            this.messageQueue.enqueue(message, MessagePriority.CRITICAL)
            return success(undefined)
          },
          (error) => failure(error),
          interruptResult
        )
        
      default:
        console.log(`❓ Unknown hotkey: ${key}`)
    }
    
    return success(undefined)
  }
  
  getMessageQueue(): QiAsyncMessageQueue<QiMessage> {
    return this.messageQueue
  }
  
  getStateManager(): CLIStateManager {
    return this.stateManager
  }
  
  getStats(): any {
    return {
      isRunning: this.isRunning,
      sessionId: this.sessionId,
      currentState: `${this.stateManager.getCurrentState()}${this.stateManager.getCurrentState() === 'ready' ? `.${this.stateManager.getCurrentSubState()}` : ''}`,
      queueStats: this.messageQueue.getStats()
    }
  }
}

// Demo the message-driven CLI
console.log('🎬 Message-Driven CLI Demo\n')

const cli = new MessageDrivenCLI()

// Start the CLI
await match(
  async () => {
    console.log('✅ CLI started successfully\n')
    
    // Simulate user interactions
    console.log('--- Simulating User Interactions ---')
    
    // CLI command
    await cli.processInput('/status')
    
    // Agent input
    await cli.processInput('Can you help me build a React component?')
    
    // Simulate agent response
    await cli.receiveAgentResponse('I\'d be happy to help you build a React component! Let me create a functional component with TypeScript for you.')
    
    // Test hotkeys
    console.log('\n--- Testing Hotkeys ---')
    cli.handleHotkey('Shift+Tab')
    cli.handleHotkey('Shift+Tab')
    cli.handleHotkey('Escape')
    
    // Another agent input in different mode
    await cli.processInput('Now help me test this component')
    cli.handleHotkey('Ctrl+C') // Interrupt
    
    // Show final stats
    console.log('\n--- Final CLI Statistics ---')
    const stats = cli.getStats()
    console.log('📊 CLI Stats:')
    console.log(`   Running: ${stats.isRunning}`)
    console.log(`   Session: ${stats.sessionId}`)
    console.log(`   State: ${stats.currentState}`)
    console.log(`   Queue: ${stats.queueStats.messagesProcessed} processed, ${cli.getMessageQueue().size()} pending`)
  },
  async (error) => {
    console.error('❌ CLI startup failed:', error.message)
  },
  await cli.start()
)

Deno.jupyter.html`
<div style="background: #f8f9fa; padding: 15px; border-left: 4px solid #17a2b8; margin: 15px 0;">
  <h4 style="color: #117a8b; margin: 0 0 10px 0;">📡 Message Integration Benefits</h4>
  <ul style="margin: 10px 0; padding-left: 20px; color: #117a8b;">
    <li><strong>Responsive CLI</strong> - Local commands execute immediately</li>
    <li><strong>Async agent processing</strong> - Non-blocking AI interactions</li>
    <li><strong>State awareness</strong> - Agent receives CLI context</li>
    <li><strong>Interrupt handling</strong> - Ctrl+C sends high-priority messages</li>
    <li><strong>Session tracking</strong> - All interactions linked to session</li>
  </ul>
</div>
`

## 4. UI Components and Framework Adapters

The CLI framework supports different UI approaches through framework adapters:

In [ ]:
console.log('\n=== UI Components and Framework Adapters ===\n')

// Mode indicator component
class ModeIndicator {
  private static modeConfig = {
    ready: {
      planning: { emoji: '🧠', color: 'blue', label: 'Planning' },
      editing: { emoji: '✏️', color: 'green', label: 'Editing' },
      generic: { emoji: '💬', color: 'white', label: 'Generic' }
    },
    busy: { emoji: '⚙️', color: 'yellow', label: 'Processing' }
  }
  
  static render(state: AppState, subState: AppSubState, taskName?: string | null): string {
    if (state === 'busy') {
      const config = this.modeConfig.busy
      const task = taskName ? ` ${taskName}` : ''
      return `${config.emoji} ${config.label}${task}`
    } else {
      const config = this.modeConfig.ready[subState]
      return `${config.emoji} ${config.label}`
    }
  }
  
  static getStatusLine(state: AppState, subState: AppSubState, sessionId: string): string {
    const mode = this.render(state, subState)
    const timestamp = new Date().toLocaleTimeString()
    return `[${timestamp}] ${mode} | Session: ${sessionId.substring(0, 12)}...`
  }
}

// Progress display component
class ProgressDisplay {
  private static spinnerFrames = ['⠋', '⠙', '⠹', '⠸', '⠼', '⠴', '⠦', '⠧', '⠇', '⠏']
  private frameIndex = 0
  
  renderSpinner(message: string): string {
    const frame = ProgressDisplay.spinnerFrames[this.frameIndex]
    this.frameIndex = (this.frameIndex + 1) % ProgressDisplay.spinnerFrames.length
    return `${frame} ${message}`
  }
  
  renderProgress(current: number, total: number, message: string): string {
    const percentage = Math.round((current / total) * 100)
    const barLength = 20
    const filledLength = Math.round((current / total) * barLength)
    const bar = '█'.repeat(filledLength) + '░'.repeat(barLength - filledLength)
    
    return `[${bar}] ${percentage}% ${message} (${current}/${total})`
  }
}

// Framework adapter interface (conceptual)
interface ICLIFramework {
  initialize(): Promise<Result<void, QiError>>
  start(): Promise<Result<void, QiError>>
  stop(): Promise<Result<void, QiError>>
  updateDisplay(state: AppState, subState: AppSubState, message?: string): void
  getFrameworkName(): string
}

// Readline-based framework adapter (conceptual)
class ReadlineFramework implements ICLIFramework {
  private isInitialized = false
  private isStarted = false
  
  async initialize(): Promise<Result<void, QiError>> {
    console.log('🔧 ReadlineFramework: Initializing...')
    // Would set up Node.js readline interface here
    this.isInitialized = true
    return success(undefined)
  }
  
  async start(): Promise<Result<void, QiError>> {
    if (!this.isInitialized) {
      return failure(systemError('Framework not initialized'))
    }
    
    console.log('🚀 ReadlineFramework: Started')
    console.log('   Features: Basic terminal input/output')
    console.log('   Hotkeys: Limited (Ctrl+C only)')
    console.log('   UI: Text-based prompts')
    
    this.isStarted = true
    return success(undefined)
  }
  
  async stop(): Promise<Result<void, QiError>> {
    console.log('🛑 ReadlineFramework: Stopping...')
    this.isStarted = false
    return success(undefined)
  }
  
  updateDisplay(state: AppState, subState: AppSubState, message?: string): void {
    const statusLine = ModeIndicator.getStatusLine(state, subState, 'readline_session')
    console.log(`\n${statusLine}`)
    if (message) {
      console.log(`📄 ${message}`)
    }
  }
  
  getFrameworkName(): string {
    return 'Readline'
  }
}

// Hybrid framework adapter (conceptual - combines readline with custom rendering)
class HybridFramework implements ICLIFramework {
  private isInitialized = false
  private isStarted = false
  private progressDisplay = new ProgressDisplay()
  
  async initialize(): Promise<Result<void, QiError>> {
    console.log('🔧 HybridFramework: Initializing...')
    // Would set up readline + custom ANSI rendering here
    this.isInitialized = true
    return success(undefined)
  }
  
  async start(): Promise<Result<void, QiError>> {
    if (!this.isInitialized) {
      return failure(systemError('Framework not initialized'))
    }
    
    console.log('🚀 HybridFramework: Started')
    console.log('   Features: Enhanced terminal with custom rendering')
    console.log('   Hotkeys: Full support (Shift+Tab, Escape, Ctrl+C)')
    console.log('   UI: Rich text, progress bars, status indicators')
    
    this.isStarted = true
    return success(undefined)
  }
  
  async stop(): Promise<Result<void, QiError>> {
    console.log('🛑 HybridFramework: Stopping...')
    this.isStarted = false
    return success(undefined)
  }
  
  updateDisplay(state: AppState, subState: AppSubState, message?: string): void {
    // Simulate rich UI rendering
    console.log('\n' + '═'.repeat(60))
    
    const modeDisplay = ModeIndicator.render(state, subState)
    console.log(`┃ Mode: ${modeDisplay.padEnd(20)} ┃ Session: hybrid_session   ┃`)
    
    if (state === 'busy' && message) {
      const spinner = this.progressDisplay.renderSpinner(message)
      console.log(`┃ ${spinner.padEnd(52)} ┃`)
    } else if (message) {
      console.log(`┃ ${message.padEnd(52)} ┃`)
    }
    
    console.log('═'.repeat(60))
    
    if (state === 'ready') {
      console.log(`💡 Tip: Press Shift+Tab to cycle modes (current: ${subState})`)
    }
  }
  
  getFrameworkName(): string {
    return 'Hybrid'
  }
}

// Demonstrate UI components and framework adapters
console.log('🎬 UI Components and Framework Adapters Demo\n')

// Test mode indicator
console.log('--- Mode Indicator Component ---')
console.log('Ready states:')
console.log(`   ${ModeIndicator.render('ready', 'generic')}`)
console.log(`   ${ModeIndicator.render('ready', 'planning')}`)
console.log(`   ${ModeIndicator.render('ready', 'editing')}`)
console.log('Busy state:')
console.log(`   ${ModeIndicator.render('busy', 'generic', 'Building component')}`)

// Test progress display
console.log('\n--- Progress Display Component ---')
const progress = new ProgressDisplay()
console.log('Spinner animations:')
for (let i = 0; i < 5; i++) {
  console.log(`   ${progress.renderSpinner('Loading...')}`)
}

console.log('\nProgress bars:')
for (let i = 0; i <= 10; i += 3) {
  console.log(`   ${progress.renderProgress(i, 10, 'Processing files')}`)
}

// Test framework adapters
console.log('\n--- Framework Adapters ---\n')

const frameworks = [new ReadlineFramework(), new HybridFramework()]

for (const framework of frameworks) {
  console.log(`Testing ${framework.getFrameworkName()} Framework:`)
  
  await match(
    async () => {
      await match(
        async () => {
          // Simulate state changes
          framework.updateDisplay('ready', 'generic', 'Welcome to QiCore CLI')
          framework.updateDisplay('ready', 'planning', 'Switched to planning mode')
          framework.updateDisplay('busy', 'generic', 'Processing your request')
          framework.updateDisplay('ready', 'editing', 'Task completed')
          
          await framework.stop()
        },
        async (error) => console.error(`❌ Framework error: ${error.message}`),
        await framework.start()
      )
    },
    async (error) => console.error(`❌ Initialization error: ${error.message}`),
    await framework.initialize()
  )
  
  console.log('\n' + '-'.repeat(50) + '\n')
}

Deno.jupyter.html`
<div style="background: #e3f2fd; padding: 15px; border-radius: 8px; margin: 15px 0;">
  <h4 style="color: #1565c0; margin: 0 0 10px 0;">🎨 UI Framework Benefits</h4>
  <ul style="margin: 10px 0; padding-left: 20px; color: #1565c0;">
    <li><strong>Pluggable architecture</strong> - Switch between UI frameworks</li>
    <li><strong>Progressive enhancement</strong> - From basic to rich terminal UI</li>
    <li><strong>Consistent state display</strong> - Mode indicators across frameworks</li>
    <li><strong>Interactive feedback</strong> - Progress bars and spinners</li>
    <li><strong>Hotkey support</strong> - Framework-specific input handling</li>
  </ul>
</div>
`

## 5. Complete Integration Example

Let's bring everything together in a complete CLI application example:

In [ ]:
console.log('\n=== Complete CLI Application Demo ===\n')

// Complete CLI application that integrates all components
class CompleteCLIApplication {
  private cli: MessageDrivenCLI
  private framework: ICLIFramework
  private isRunning = false
  private sessionStats = {
    startTime: 0,
    commandsExecuted: 0,
    agentInteractions: 0,
    modeChanges: 0
  }
  
  constructor(frameworkType: 'readline' | 'hybrid' = 'hybrid') {
    this.cli = new MessageDrivenCLI()
    this.framework = frameworkType === 'readline' 
      ? new ReadlineFramework() 
      : new HybridFramework()
    
    // Set up state manager subscription
    this.cli.getStateManager().subscribe((state, subState) => {
      this.framework.updateDisplay(state, subState)
      if (state === 'ready') {
        this.sessionStats.modeChanges++
      }
    })
    
    console.log(`🏗️ Complete CLI Application created with ${this.framework.getFrameworkName()} framework`)
  }
  
  async initialize(): Promise<Result<void, QiError>> {
    console.log('🔧 Initializing complete CLI application...')
    
    // Initialize framework
    const frameworkResult = await this.framework.initialize()
    if (!match(() => true, () => false, frameworkResult)) {
      return frameworkResult
    }
    
    // Initialize CLI
    const cliResult = await this.cli.start()
    if (!match(() => true, () => false, cliResult)) {
      return cliResult
    }
    
    console.log('✅ Complete CLI application initialized')
    return success(undefined)
  }
  
  async start(): Promise<Result<void, QiError>> {
    if (this.isRunning) {
      return failure(systemError('Application already running'))
    }
    
    const frameworkResult = await this.framework.start()
    if (!match(() => true, () => false, frameworkResult)) {
      return frameworkResult
    }
    
    this.isRunning = true
    this.sessionStats.startTime = Date.now()
    
    console.log('\n🚀 Complete CLI Application started!')
    console.log('\n💡 Available interactions:')
    console.log('   • Type /help for CLI commands')
    console.log('   • Type anything else to interact with AI agent')
    console.log('   • Press Shift+Tab to cycle modes')
    console.log('   • Press Ctrl+C to interrupt operations')
    
    this.framework.updateDisplay('ready', 'generic', 'Application ready for input')
    
    return success(undefined)
  }
  
  async processInput(input: string): Promise<Result<void, QiError>> {
    if (!this.isRunning) {
      return failure(systemError('Application not running'))
    }
    
    // Update stats
    if (input.startsWith('/')) {
      this.sessionStats.commandsExecuted++
    } else {
      this.sessionStats.agentInteractions++
    }
    
    return await this.cli.processInput(input)
  }
  
  handleHotkey(key: string): Result<void, QiError> {
    return this.cli.handleHotkey(key)
  }
  
  async simulateAgentResponse(response: string): Promise<Result<void, QiError>> {
    return await this.cli.receiveAgentResponse(response)
  }
  
  getSessionStats(): any {
    const runtime = this.sessionStats.startTime > 0 
      ? Math.round((Date.now() - this.sessionStats.startTime) / 1000)
      : 0
    
    return {
      ...this.sessionStats,
      runtime,
      framework: this.framework.getFrameworkName(),
      cliStats: this.cli.getStats()
    }
  }
  
  async stop(): Promise<Result<void, QiError>> {
    console.log('\n🛑 Stopping complete CLI application...')
    
    this.isRunning = false
    
    const frameworkResult = await this.framework.stop()
    if (!match(() => true, () => false, frameworkResult)) {
      return frameworkResult
    }
    
    console.log('✅ Complete CLI application stopped')
    return success(undefined)
  }
}

// Demo the complete CLI application
console.log('🎬 Complete CLI Application Demo\n')

const app = new CompleteCLIApplication('hybrid')

await match(
  async () => {
    await match(
      async () => {
        console.log('\n--- Simulating User Session ---')
        
        // Welcome and help
        await app.processInput('/help')
        
        // Check status
        await app.processInput('/status')
        
        // Switch to planning mode
        app.handleHotkey('Shift+Tab')
        
        // Agent interaction in planning mode
        await app.processInput('I need to design a user authentication system')
        await app.simulateAgentResponse('I\'ll help you design a secure authentication system. Let me outline the key components: JWT tokens, password hashing, session management, and security middleware.')
        
        // Switch to editing mode
        app.handleHotkey('Shift+Tab')
        
        // Agent interaction in editing mode
        await app.processInput('Now implement the JWT token service')
        await app.simulateAgentResponse('Here\'s a TypeScript JWT service implementation with proper error handling using Result<T> patterns...')
        
        // Test interrupt
        await app.processInput('Generate comprehensive tests for the auth system')
        app.handleHotkey('Ctrl+C') // Interrupt while busy
        
        // Final status check
        await app.processInput('/version')
        
        console.log('\n--- Session Statistics ---')
        const stats = app.getSessionStats()
        console.log('📊 Session Summary:')
        console.log(`   Runtime: ${stats.runtime} seconds`)
        console.log(`   CLI Commands: ${stats.commandsExecuted}`)
        console.log(`   Agent Interactions: ${stats.agentInteractions}`)
        console.log(`   Mode Changes: ${stats.modeChanges}`)
        console.log(`   Framework: ${stats.framework}`)
        console.log(`   Current State: ${stats.cliStats.currentState}`)
        
        await app.stop()
      },
      async (error) => {
        console.error('❌ Application startup failed:', error.message)
        await app.stop()
      },
      await app.start()
    )
  },
  async (error) => {
    console.error('❌ Application initialization failed:', error.message)
  },
  await app.initialize()
)

Deno.jupyter.html`
<div style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; padding: 20px; border-radius: 10px; margin: 20px 0; text-align: center;">
  <h3 style="margin: 0 0 15px 0;">🏆 Complete CLI Integration Success!</h3>
  <p style="margin: 0; font-size: 16px;">
    You've seen a full CLI framework with state management, message integration, UI components, and production-ready patterns!
  </p>
</div>
`

## 🎯 Key Takeaways

After completing this tutorial, you now understand:

### ✅ **State Machine Architecture**
- **Hierarchical states** (busy/ready → planning/editing/generic)
- **XState 5 patterns** for predictable state transitions
- **Mode cycling** with Shift+Tab for different interaction contexts
- **Task lifecycle** management for busy/ready states

### ✅ **Command System Design**
- **CLI commands** (start with /) handled locally for immediate response
- **Agent commands** (everything else) sent via message queue
- **Built-in commands** (help, exit, clear, version, status)
- **Alias support** for user convenience

### ✅ **Message-Driven Integration**
- **Pure message architecture** - no direct CLI ↔ Agent coupling
- **Session context** passed with messages for state awareness
- **Priority handling** - interrupts processed immediately
- **Request-response correlation** for interaction tracking

### ✅ **UI Framework Adapters**
- **Pluggable architecture** - switch between UI frameworks
- **Readline framework** for basic terminal interfaces
- **Hybrid framework** for enhanced UI with progress indicators
- **Consistent components** across different frameworks

### ✅ **Production Patterns**
- **Hotkey handling** (Shift+Tab, Escape, Ctrl+C)
- **Visual feedback** with mode indicators and progress displays
- **Error handling** with Result<T> patterns throughout
- **Statistics tracking** for monitoring and analytics
- **Graceful shutdown** and resource cleanup

## 🚀 Next Steps

Congratulations! You've completed all QiCore tutorials. Here's how to continue your journey:

### 📖 **Review and Practice**
- **[01-qi-base.ipynb](./01-qi-base.ipynb)** - Master Result<T> patterns
- **[02-qi-core.ipynb](./02-qi-core.ipynb)** - Build robust services
- **[03-qi-amsg.ipynb](./03-qi-amsg.ipynb)** - Design message-driven systems

### 🛠️ **Try Real Examples**
- Explore all working examples in `typescript/app/`
- Run the CLI-AMSG integration example
- Build your own CLI application using these patterns

### 📚 **Dive Deeper**
- Read the complete [API Documentation](../api/README.md)
- Study the architectural guides in `docs/`
- Learn about XState 5 and advanced state machine patterns
- Explore h2A patterns and async iteration best practices

### 🏗️ **Build Production Applications**
- Use QiCore as the foundation for your TypeScript projects
- Implement proper error handling with Result<T> patterns
- Design message-driven architectures for scalability
- Build sophisticated CLI tools with state management

**Remember**: QiCore provides a complete foundation for building reliable, scalable, and maintainable TypeScript applications. The patterns you've learned here will serve you well in any project!

In [ ]:
// Final challenge: Design your own CLI application architecture!
// Here's a template that combines all QiCore concepts

interface MyAppArchitecture {
  // Foundation layer
  base: {
    errorHandling: 'Result<T, QiError>';
    functionalComposition: 'map, flatMap, match';
    asyncPatterns: 'Promise<Result<T>>';
  };
  
  // Infrastructure layer
  core: {
    configuration: 'Multi-source with validation';
    logging: 'Structured with context';
    caching: 'Memory or Redis backends';
  };
  
  // Communication layer
  amsg: {
    messageQueue: 'Priority-based async queue';
    messageTypes: 'Typed message system';
    h2aPatterns: 'Async iteration over messages';
  };
  
  // Interface layer
  cli: {
    stateManagement: 'XState hierarchical states';
    commandSystem: 'CLI vs Agent command separation';
    uiFramework: 'Pluggable UI adapters';
    hotkeySupport: 'Full keyboard interaction';
  };
}

console.log('\n🎨 Your QiCore Application Architecture Template:')
console.log()
console.log('1. 🏗️ Foundation (qi/base)')
console.log('   → Result<T> for all operations that can fail')
console.log('   → Functional composition with map/flatMap/match')
console.log('   → Structured error categories for retry strategies')
console.log()
console.log('2. 🔧 Infrastructure (qi/core)')
console.log('   → Configuration from multiple sources with validation')
console.log('   → Structured logging with context accumulation')
console.log('   → Unified caching with multiple backends')
console.log()
console.log('3. 📨 Communication (qi/amsg)')
console.log('   → Priority-based message queue for component communication')
console.log('   → Comprehensive message type system')
console.log('   → h2A async iteration patterns for message processing')
console.log()
console.log('4. 🖥️ Interface (qi/cli)')
console.log('   → State machine for interaction mode management')
console.log('   → Command system with CLI/Agent separation')
console.log('   → Pluggable UI framework adapters')
console.log('   → Full hotkey and keyboard interaction support')
console.log()
console.log('💡 Key Benefits:')
console.log('   ✅ Type-safe error handling eliminates crashes')
console.log('   ✅ Functional composition enables reliable operations')
console.log('   ✅ Message-driven architecture ensures scalability')
console.log('   ✅ State management provides predictable behavior')
console.log('   ✅ Production-ready patterns for real applications')

Deno.jupyter.html`
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 25px 0; text-align: center;">
  <h2 style="margin: 0 0 20px 0; font-size: 28px;">🎉 QiCore Tutorial Series Complete!</h2>
  <p style="margin: 0 0 15px 0; font-size: 18px;">
    You've mastered the complete QiCore ecosystem:
  </p>
  <div style="display: flex; justify-content: center; gap: 20px; margin: 20px 0; flex-wrap: wrap;">
    <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 120px;">
      <div style="font-size: 24px; margin-bottom: 5px;">🏗️</div>
      <div style="font-size: 16px; font-weight: bold;">Base</div>
      <div style="font-size: 12px;">Result&lt;T&gt;</div>
    </div>
    <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 120px;">
      <div style="font-size: 24px; margin-bottom: 5px;">🔧</div>
      <div style="font-size: 16px; font-weight: bold;">Core</div>
      <div style="font-size: 12px;">Services</div>
    </div>
    <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 120px;">
      <div style="font-size: 24px; margin-bottom: 5px;">📨</div>
      <div style="font-size: 16px; font-weight: bold;">AMSG</div>
      <div style="font-size: 12px;">Messages</div>
    </div>
    <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; min-width: 120px;">
      <div style="font-size: 24px; margin-bottom: 5px;">🖥️</div>
      <div style="font-size: 16px; font-weight: bold;">CLI</div>
      <div style="font-size: 12px;">Interface</div>
    </div>
  </div>
  <p style="margin: 20px 0 0 0; font-size: 16px; font-weight: bold;">
    Now go build amazing, reliable TypeScript applications! 🚀
  </p>
</div>
`